In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

## FinanaceBert

In [3]:
# initialize tokenizer and model from pretrained model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [5]:
def sentiment(tokens):
    # get output logits from model
    output = model(**tokens)
    # get probabilities from logits
    probs = torch.nn.functional.softmax(output.logits, dim=-1)
    # return probabilities
    return probs

In [11]:
txt = """
I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought.”
"""

In [12]:
tokens = tokenizer.encode_plus(txt,add_special_tokens=False)
len(tokens['input_ids'])

1345

# Splittiing the text into chunks of 512 tokens

In [14]:
type(tokens['input_ids'])

list

In [15]:
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [16]:
input_ids[16:32]

[1045,
 2572,
 2025,
 5191,
 2055,
 1996,
 3006,
 2091,
 22299,
 2021,
 1996,
 5573,
 3623,
 1999,
 16189,
 1012]

## Window method

In [19]:

probs_list = []

start = 0
window_size = 510
total_len = len(input_ids)
loop = True

while loop:
    end = start + window_size
    if end >= total_len:
        end = total_len
        loop = False
    
    #extract window from input_ids and attention_mask
    input_ids_chunk = [101] + input_ids[start:end] + [102]
    attention_mask_chunk = [1] + attention_mask[start:end] + [1]
    input_ids_chunk += [0] * (window_size - len(input_ids_chunk))
    attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk))

    input_dict ={
        'input_ids': torch.tensor([input_ids_chunk]).long(),
        'attention_mask': torch.tensor([attention_mask_chunk]).int()
    }

    # get probabilities from model
    probs_list.append(sentiment(input_dict))

    print(start, end)
    start = end


0 510
510 1020
1020 1345


In [20]:
stack = torch.stack(probs_list)
stack.shape

torch.Size([3, 1, 3])

In [21]:
with torch.no_grad():
    stack = torch.stack(probs_list)
    stack = stack.resize_(stack.shape[0], stack.shape[2])
    avg =stack.mean(dim=0)

avg    

tensor([0.4144, 0.4940, 0.0916])

In [23]:
torch.argmax(avg).item()

1

In [30]:
tokens = tokenizer.encode_plus(txt,add_special_tokens=True, max_length=512, truncation=True, return_tensors='pt', padding='max_length')

In [31]:
tokens

{'input_ids': tensor([[  101,  1045,  2052,  2066,  2000,  2131,  2115,  2035,  4301,  2006,
          1996,  5416, 10750,  3623,  2023,  2733,  1012,  1045,  2572,  2025,
          5191,  2055,  1996,  3006,  2091, 22299,  2021,  1996,  5573,  3623,
          1999, 16189,  1012,  2006,  1016,  1013,  2385,  1996,  2184,  2095,
          9547, 16189,  3445,  2011,  2471,  1023,  3867,  1998,  2006,  1016,
          1013,  2539,  1996, 10750,  3445,  2011,  2471,  1019,  3867,  1012,
          3145,  2685,  2013,  1996, 27166,  9818,  3720,  1024,  1008,  1008,
          1008,  1996,  1523,  6823,  2099,  9092, 24456,  1524,  1999,  2286,
          2001,  1037,  5573,  9997,  1999,  9837, 16189,  2349,  2000,  3006,
          6634,  2044,  1996,  2976,  3914,  2623,  2008,  2009,  2052,  4088,
          6823,  4892,  2049, 20155, 24070,  2565,  1012,  1008,  1008,  1008,
          1008,  1008,  2350,  2430,  5085,  2105,  1996,  2088,  2031,  3013,
          3037,  6165,  2000,  3181,  

In [32]:
probs = sentiment(tokens)

In [33]:
probs

tensor([[0.1384, 0.8145, 0.0471]], grad_fn=<SoftmaxBackward0>)

In [34]:
torch.argmax(probs).item()

1